In [1]:
import pandas as pd
from utils.numba_utils import *
from utils.variable_utils import *
from gc import collect as gc
import warnings
warnings.filterwarnings('ignore')

In [2]:
ticker = pd.read_parquet('./data/BTCUSDT-bookTicker-2023-10-01.p').to_records(index=False)
trades = pd.read_parquet('./data/BTCUSDT-aggTrades-2023-10-01.p').to_records(index=False)

In [3]:
from variables import feature, target
feature_names = [_ for _ in dir(feature) if _.startswith('feature_')]
target_names = [_ for _ in dir(target) if _.startswith('target_')]
feature_names

['feature_AutoCov',
 'feature_Breadth',
 'feature_EffectiveSpread',
 'feature_Immediacy',
 'feature_Lambda',
 'feature_LobImbalance',
 'feature_PastReturn',
 'feature_QuotedSpread',
 'feature_TxnImbalance',
 'feature_VolumeAll',
 'feature_VolumeAvg',
 'feature_VolumeMax']

In [4]:
funcs = [getattr(feature, _) for _ in feature_names]
get_names = lambda delta1, delta2: [f'{n}_{delta1}_{delta2}' for n in feature_names]
deltas = [(0, 50), (50, 200)]
dfs = []
for d1, d2 in deltas:
    dfs.append(backward_apply(ticker, trades, 'trading', d1, d2, funcs, get_names(d1, d2)))
df = pd.concat(dfs, axis=1)

100%|██████████| 12/12 [00:27<00:00,  2.29s/it]


In [5]:
target_names

['target_last_mp', 'target_last_txn_p', 'target_mean_txn_p']

In [6]:
# calculate the mean price of future 20 trades. used as prediction target
future_fairpx = forward_apply(ticker, trades, 'trading', 20, target.target_mean_txn_p, 'target_mean_txn_p')
future_rtn = log(future_fairpx.values / ticker['mp']) * 1e4 # future return in bps

In [7]:
df['future_rtn'] = future_rtn
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_parquet('./data/df.p')